In [1]:
from nestedtensor import torch
from IPython.display import Markdown, display
def print_eval(s):
    colorS = "<span style='color:darkred'>$ {}</span>".format(s)
    display(Markdown('**{}**'.format(colorS))) 
    print('{}\n'.format(str(eval(s))))

In [2]:
import time as time_module
def time(fn):
    t0 = time_module.time()
    count = 0
    past = 0
    while past < 10.0:
        fn()
        past = time_module.time() - t0
        count += 1
    past = past / count
    return "average {:2.4f}ms based on {} samples".format(past * 1000, count)

In [3]:
def generate_tensors(num_tensor, vocab_size):
    sentence_lengths = torch.normal(75.0, 10.0, size=(num_tensor,)).long()
    return [(torch.rand(l) * vocab_size).long() for l in sentence_lengths]

def generate_text(text):
    offsets = [0] + [len(entry) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text.to(torch.int64), offsets

class TextSentiment(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = torch.nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        emb = self.embedding(text, offsets)
        return self.fc(emb)
    
vocab_size = 10000
model = TextSentiment(10000, 256, 5)
tensors = generate_tensors(16, 10000)
text, offsets = generate_text(tensors)
nt_text = torch.nested_tensor(tensors)

print_eval("time(lambda: model(text, offsets))")
print_eval("time(lambda: model(nt_text, None))")

**<span style='color:darkred'>$ time(lambda: model(text, offsets))</span>**

average 0.3052ms based on 32763 samples



**<span style='color:darkred'>$ time(lambda: model(nt_text, None))</span>**

average 0.4228ms based on 23652 samples



In [4]:
from torchvision import models

model = models.resnet18(pretrained=False)
images = torch.rand(128, 3, 40, 50)
print_eval("images.numel()")
print_eval("time(lambda: model(images))")

nested_images = torch.nested_tensor(torch.rand(128, 3, 40, 50).unbind())
print_eval("time(lambda: model(nested_images))")

# There is still about a 10x gap in performance, which however
# can be significantly allieviated via custom code (e.g. using im2col).
images = [torch.rand(3, (i * 16) % 40 + 40, (i * 16) % 50 + 40) for i in range(64)]
nested_irregular_images = torch.nested_tensor(images)
print_eval("nested_irregular_images.numel()")
print_eval("nested_irregular_images.size()")
print_eval("time(lambda: model(nested_irregular_images))")

**<span style='color:darkred'>$ images.numel()</span>**

768000



**<span style='color:darkred'>$ time(lambda: model(images))</span>**

average 75.3909ms based on 133 samples



**<span style='color:darkred'>$ time(lambda: model(nested_images))</span>**

...

692112



**<span style='color:darkred'>$ nested_irregular_images.size()</span>**

(64, 3, None, None)



**<span style='color:darkred'>$ time(lambda: model(nested_irregular_images))</span>**

average 2866.2733ms based on 4 samples



In [5]:
def generate_tensors(num_tensor, num_features):
    sentence_lengths = torch.normal(75.0, 10.0, size=(num_tensor,)).long()
    return [torch.rand(l.item(), num_features) for l in sentence_lengths]

tensors = generate_tensors(32, 256)
nt_text = torch.nested_tensor(tensors)
text = torch.rand(32, 75, 256)

h0 = torch.randn(6, len(nt_text), 512)
c0 = torch.randn(6, len(nt_text), 512)
print_eval("nt_text.nested_size(1)")
print_eval("nt_text.numel()")
print_eval("text.numel()")
print_eval("time(lambda: torch.nn.LSTM(256, 512, 6, batch_first=True)(nt_text, (h0, c0)))")
print_eval("time(lambda: torch.nn.LSTM(256, 512, 6, batch_first=True)(text, (h0, c0)))")

**<span style='color:darkred'>$ nt_text.nested_size(1)</span>**

(74, 59, 68, 67, 85, 81, 75, 68, 74, 69, 80, 70, 78, 75, 91, 89, 59, 61, 75, 60, 73, 75, 71, 66, 73, 55, 78, 59, 75, 55, 70, 87)



**<span style='color:darkred'>$ nt_text.numel()</span>**

587520



**<span style='color:darkred'>$ text.numel()</span>**

614400



**<span style='color:darkred'>$ time(lambda: torch.nn.LSTM(256, 512, 6, batch_first=True)(nt_text, (h0, c0)))</span>**

average 2431.1809ms based on 5 samples



**<span style='color:darkred'>$ time(lambda: torch.nn.LSTM(256, 512, 6, batch_first=True)(text, (h0, c0)))</span>**

average 518.9265ms based on 20 samples

